
The R0 is between 2 and 6, meaning each infected person infects 2–6 others, which can cause the number of infected to double every few days.

In [ ]:

# https://informationisbeautiful.net/visualizations/covid-19-coronavirus-infographic-datapack/
# https://docs.google.com/spreadsheets/d/1g_YxmDfQx7aOU2DKzNZo9b-NTk62Bju6X3z6OuCa6gw/edit#gid=515684451

In [ ]:

command_str = '{sys.executable} -m pip install --upgrade statsmodels'.format(sys=sys)
print(command_str)
!{command_str}

In [1]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [2]:

%pprint
%run ../../load_magic/storage.py
%run ../../load_magic/paths.py
%run ../../load_magic/lists.py
%run ../../load_magic/environment.py
%run ../../load_magic/dataframes.py

import scraping_utils
wikipedia = scraping_utils.wikipedia

from bs4 import BeautifulSoup as bs
import networkx as nx
import re
import urllib.request

notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
#pandemic_df = s.load_object('pandemic_df')
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])

dir()

Pretty printing has been turned OFF
D:\Documents\Repositories\notebooks\covid19\ipynb\Corona Virus Dataset Creation.ipynb
['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']


['Config', 'In', 'Out', 'RandomForestClassifier', 'SequenceMatcher', 'Storage', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '__warningregistry__', '_dh', '_i', '_i1', '_i2', '_ih', '_ii', '_iii', '_oh', 'bs', 'check_4_doubles', 'check_for_typos', 'conjunctify_list', 'copyfile', 'csv', 'encoding', 'example_iterrows', 'exit', 'filepath_regex', 'get_classifier', 'get_column_descriptions', 'get_data_structs_dataframe', 'get_datastructure_prediction', 'get_dir_tree', 'get_git_lfs_track_commands', 'get_importances', 'get_input_sample', 'get_ipython', 'get_max_rsquared_adj', 'get_module_version', 'get_notebook_path', 'get_page_tables', 'get_specific_gitignore_files', 'get_struct_name', 'humanize_bytes', 'io', 'ipykernel', 'json', 'jupyter_config_dir', 'math', 'notebook_path', 'notebookapp', 'nx', 'os', 'pd', 'pickle', 'plt', 'preprocess_data', 'print_all_files_ending_starting_with', 'print_all_files_ending_wit

In [ ]:

!start %windir%\explorer.exe "{os.path.abspath(os.path.dirname(notebook_path))}"


---
# Scrape the Data

In [7]:

tables_url = 'https://en.wikipedia.org/wiki/ISO_3166-1'
tables_list = get_page_tables(tables_url)

[(1, (249, 6)), (2, (36, 3)), (0, (6, 7)), (3, (7, 2)), (4, (2, 2))]


In [12]:

country_abbreviations_df = tables_list[1].copy()
country_abbreviations_df.set_index('English short name (using title case)', drop=True, inplace=True)
country_abbreviations_dict = country_abbreviations_df['Alpha-2 code'].to_dict()
s.store_objects(country_abbreviations_dict=country_abbreviations_dict)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\country_abbreviations_dict.pickle


In [ ]:

file_path = os.path.join(s.data_csv_folder, 'Confirmed_Cumulative.csv')
world_cumulative_df = s.load_csv('Confirmed_Cumulative')
date_format = '%b %d, %Y'
for column_name in world_cumulative_df.columns:
    if column_name == 'Date':
        world_cumulative_df[column_name] = world_cumulative_df[column_name].map(lambda x: datetime.strptime('{}'.format(x), date_format))
        world_cumulative_df[column_name] = pd.to_datetime(world_cumulative_df[column_name])
        pass
    else:
        world_cumulative_df[column_name] = world_cumulative_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        world_cumulative_df[column_name] = pd.to_numeric(world_cumulative_df[column_name], errors='coerce')
s.store_objects(world_cumulative_df=world_cumulative_df)

In [8]:

from datetime import datetime

url = 'https://docs.google.com/spreadsheets/u/2/d/e/2PACX-1vRwAqp96T9sYYq2-i7Tj0pvTf6XVHjDSMIKBdZHXiCGGdNC0ypEU9NbngS8mxea55JuCFuua1MUeOj5/pubhtml'
tables_list = get_page_tables(url)

[(3, (2622, 14)), (1, (58, 15)), (4, (56, 14)), (5, (58, 6)), (0, (12, 3)), (2, (2, 12))]


In [14]:

tables_list[2].copy()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,1,Positive,Negative,Pending,HospitalizedÂ â Currently,Hospitalized â Cumulative,In ICU âÂ Currently,In ICU âÂ Cumulative,On Ventilator âÂ Currently,On Ventilator âÂ Cumulative,Recovered,Deaths
1,2,799717,3355461,3956,58468,84292,15228,2315,5514,214,58117,39995


In [4]:

covidtracking_df.columns = ['County_Name1', 'County_Name', 'Confirmed_Cumulative', 'Deaths_Cumulative', 'Recovered_Cumulative', 'References']
covidtracking_df = covidtracking_df.iloc[:-2]
columns_list = ['County_Name', 'Confirmed_Cumulative', 'Deaths_Cumulative', 'Recovered_Cumulative']
covidtracking_df = covidtracking_df[columns_list]
for column_name in covidtracking_df.columns:
    if column_name == 'County_Name':
        covidtracking_df[column_name] = covidtracking_df[column_name].map(lambda x: x.split('[')[0])
    else:
        covidtracking_df[column_name] = covidtracking_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('[')[0]))
        covidtracking_df[column_name] = pd.to_numeric(covidtracking_df[column_name], errors='coerce')
s.store_objects(covidtracking_df=covidtracking_df)
covidtracking_df.sort_values('Deaths_Cumulative', ascending=False).head(10)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\pandemic_countries_df.pickle


,County_Name,Confirmed_Cumulative,Deaths_Cumulative,Recovered_Cumulative
0,United States,735074.0,38568.0,63655.0
2,Italy,175927.0,23227.0,44927.0
1,Spain,192367.0,20181.0,74662.0
5,France,111821.0,19323.0,35983.0
4,United Kingdom,114217.0,15464.0,NaN
9,Belgium,37183.0,5453.0,8348.0
8,Iran,80868.0,5031.0,55987.0
6,China,82719.0,4632.0,77029.0
3,Germany,143216.0,4453.0,77147.0
13,Netherlands,31589.0,3601.0,NaN


In [60]:

from datetime import datetime

tables_url = 'https://en.wikipedia.org/wiki/2019%E2%80%9320_coronavirus_pandemic_by_country_and_territory'
tables_list = get_page_tables(tables_url)

[(0, (235, 6)), (6, (22, 6)), (9, (65, 2)), (1, (61, 2)), (7, (11, 7)), (4, (11, 6)), (2, (15, 4)), (11, (20, 2)), (12, (19, 2)), (23, (8, 2)), (24, (7, 2)), (26, (7, 2)), (3, (2, 6)), (13, (6, 2)), (17, (6, 2)), (27, (6, 2)), (18, (5, 2)), (16, (4, 2)), (19, (4, 2)), (20, (3, 2)), (21, (3, 2)), (15, (2, 2)), (22, (2, 2)), (25, (2, 2)), (5, (1, 2)), (8, (1, 2)), (10, (1, 2)), (14, (1, 2))]


In [4]:

pandemic_countries_df = tables_list[0].copy()
pandemic_countries_df.columns = ['County_Name1', 'County_Name', 'Confirmed_Cumulative', 'Deaths_Cumulative', 'Recovered_Cumulative', 'References']
pandemic_countries_df = pandemic_countries_df.iloc[:-2]
columns_list = ['County_Name', 'Confirmed_Cumulative', 'Deaths_Cumulative', 'Recovered_Cumulative']
pandemic_countries_df = pandemic_countries_df[columns_list]
for column_name in pandemic_countries_df.columns:
    if column_name == 'County_Name':
        pandemic_countries_df[column_name] = pandemic_countries_df[column_name].map(lambda x: x.split('[')[0])
    else:
        pandemic_countries_df[column_name] = pandemic_countries_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('[')[0]))
        pandemic_countries_df[column_name] = pd.to_numeric(pandemic_countries_df[column_name], errors='coerce')
s.store_objects(pandemic_countries_df=pandemic_countries_df)
pandemic_countries_df.sort_values('Deaths_Cumulative', ascending=False).head(10)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\pandemic_countries_df.pickle


,County_Name,Confirmed_Cumulative,Deaths_Cumulative,Recovered_Cumulative
0,United States,735074.0,38568.0,63655.0
2,Italy,175927.0,23227.0,44927.0
1,Spain,192367.0,20181.0,74662.0
5,France,111821.0,19323.0,35983.0
4,United Kingdom,114217.0,15464.0,NaN
9,Belgium,37183.0,5453.0,8348.0
8,Iran,80868.0,5031.0,55987.0
6,China,82719.0,4632.0,77029.0
3,Germany,143216.0,4453.0,77147.0
13,Netherlands,31589.0,3601.0,NaN


In [ ]:

for country_name in pandemic_countries_df.sort_values('Deaths_Cumulative', ascending=False).County_Name.tolist()[:5]:
    country_name = re.sub(' ', '_', str(country_name))
    rm_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_{}'.format(country_name)
    !"C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" {rm_url}

In [35]:

tables_url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'
tables_list = get_page_tables(tables_url)

[(0, (242, 6)), (1, (12, 2))]


In [36]:

country_populations_df = tables_list[0].copy()
country_populations_df.columns = ['Rank', 'Country', 'Population', 'world_population_percent', 'Date', 'Source']
country_populations_df = country_populations_df.loc[:240]
for column_name in country_populations_df.columns:
    if column_name == 'Country':
        country_populations_df[column_name] = country_populations_df[column_name].map(lambda x: str(x).split('[')[0])
    elif column_name in ['Population', 'Rank']:
        country_populations_df[column_name] = country_populations_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        country_populations_df[column_name] = pd.to_numeric(country_populations_df[column_name], errors='coerce')
country_populations_df.set_index('Country', drop=True, inplace=True)
columns_list = ['Rank', 'Population', 'Date', 'Source']
country_populations_df = country_populations_df[columns_list]
s.store_objects(country_populations_df=country_populations_df)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\country_populations_df.pickle



----
## Iran

In [3]:

page_tables_list = get_page_tables('https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Iran')

[(4, (35, 37)), (3, (61, 6)), (1, (66, 5)), (7, (66, 2)), (27, (45, 2)), (9, (21, 2)), (10, (20, 2)), (28, (14, 2)), (29, (13, 2)), (0, (12, 2)), (38, (10, 2)), (39, (9, 2)), (22, (8, 2)), (11, (7, 2)), (23, (7, 2)), (25, (7, 2)), (16, (6, 2)), (26, (6, 2)), (17, (5, 2)), (36, (5, 2)), (15, (4, 2)), (18, (4, 2)), (34, (4, 2)), (37, (4, 2)), (40, (4, 2)), (19, (3, 2)), (20, (3, 2)), (30, (3, 2)), (31, (3, 2)), (35, (3, 2)), (14, (2, 2)), (21, (2, 2)), (24, (2, 2)), (32, (2, 2)), (41, (2, 2)), (2, (1, 2)), (5, (1, 2)), (6, (1, 2)), (8, (1, 2)), (12, (1, 2)), (13, (1, 2)), (33, (1, 2))]


In [4]:

iran_df = page_tables_list[4].copy()
iran_df.columns = ['Date', 'Region_1_Qom', 'Region_1_Teh', 'Region_1_Maz', 'Region_1_Alb', 'Region_1_Sem', 'Region_1_Gol',
                   'Region_1_Qaz', 'Region_2_Esf', 'Region_2_Frs', 'Region_2_Hor', 'Region_2_Koh', 'Region_2_Cha', 'Region_2_Bus',
                   'Region_3_Gil', 'Region_3_Ard', 'Region_3_Azs', 'Region_3_Azg', 'Region_3_Kur', 'Region_3_Zan', 'Region_4_Mar',
                   'Region_4_Ham', 'Region_4_Khz', 'Region_4_Krs', 'Region_4_Lor', 'Region_4_Ilm', 'Region_5_Khr', 'Region_5_Sis',
                   'Region_5_Yaz', 'Region_5_Khs', 'Region_5_Ker', 'Region_5_Khn', 'Confirmed_New', 'Confirmed_Total',
                   'Deaths_New', 'Deaths_Total', 'Sources']
iran_df = iran_df.loc[:32]
#date_format = '%Y/%M/%d'
for column_name in iran_df.columns:
    if column_name == 'Date':
        #iran_df[column_name] = iran_df[column_name].map(lambda x: datetime.strptime('{}'.format(x), date_format))
        iran_df[column_name] = pd.to_datetime(iran_df[column_name])
        pass
    else:
        iran_df[column_name] = iran_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        iran_df[column_name] = pd.to_numeric(iran_df[column_name], errors='coerce')
s.store_objects(iran_df=iran_df)
columns_list = ['Date', 'Confirmed_Total', 'Deaths_Total', 'Confirmed_New', 'Deaths_New']
iran_df[columns_list].tail(5)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\iran_df.pickle


,Date,Confirmed_Total,Deaths_Total,Confirmed_New,Deaths_New
28,2020-03-18,17361,1135,1192,147.0
29,2020-03-19,18407,1284,1046,149.0
30,2020-03-20,19644,1433,1237,149.0
31,2020-03-21,20610,1556,966,123.0
32,2020-03-22,21638,1685,1028,129.0



----
## Sweden

In [3]:

page_tables_list = get_page_tables('https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Sweden')

[(2, (84, 37)), (1, (60, 5)), (4, (66, 2)), (6, (21, 2)), (7, (20, 2)), (0, (13, 2)), (19, (8, 2)), (8, (7, 2)), (20, (7, 2)), (22, (7, 2)), (13, (6, 2)), (23, (6, 2)), (14, (5, 2)), (12, (4, 2)), (15, (4, 2)), (16, (3, 2)), (17, (3, 2)), (11, (2, 2)), (18, (2, 2)), (21, (2, 2)), (3, (1, 2)), (5, (1, 2)), (9, (1, 2)), (10, (1, 2))]


In [23]:

sweden_df = page_tables_list[2].copy()
[re.sub(' ', '_', '{}_{}_{}'.format(column_tuple[0].split('[')[0],
                                    column_tuple[1], column_tuple[2])) for column_tuple in sweden_df.columns.tolist()]
sweden_df.columns = ['Date', 'County_Blekinge', 'County_Dalarna', 'County_Gotland', 'County_Gävleborg', 'County_Halland', 'County_Jämtland',
                     'County_Jönköping', 'County_Kalmar', 'County_Kronoberg', 'County_Norrbotten', 'County_Skåne', 'County_Stockholm',
                     'County_Södermanland', 'County_Uppsala', 'County_Värmland', 'County_Västerbotten', 'County_Västernorrland',
                     'County_Västmanland', 'County_Västra_Götaland', 'County_Örebro', 'County_Östergötland', 'Confirmed_New',
                     'Confirmed_Cumulative', 'Confirmed_Diff', 'Confirmed_7d_Ave', 'Deaths_New', 'Deaths_Cumulative', 'Deaths_Diff',
                     'Deaths_7d_Ave', 'ICU_New', 'ICU_Cumulative', 'ICU_Diff', 'ICU_7d_Ave', 'Samples_New', 'Samples_Cumulative', 'Samples_Week']
#sweden_df.Date.to_dict()
idx_list = [2] + list(range(5, 61))
mask_series = sweden_df.index.isin(idx_list)
sweden_df = sweden_df[mask_series]
columns_list = list(set(sweden_df.columns) - set(['Date', 'Confirmed_Diff', 'Deaths_Diff', 'ICU_Diff']))
for column_name in sweden_df.columns:
    if column_name == 'Date':
        sweden_df[column_name] = sweden_df[column_name].map(lambda x: str(x).split('[')[0])
        sweden_df[column_name] = pd.to_datetime(sweden_df[column_name])
        pass
    elif column_name in columns_list:
        sweden_df[column_name] = sweden_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split(' ')[0]))
        sweden_df[column_name] = pd.to_numeric(sweden_df[column_name], errors='coerce')
new_columns_list = [cn for cn in sweden_df.columns if cn.endswith('_New')]
for column_name in new_columns_list:
    sweden_df[column_name].fillna(value=0, inplace=True)
    sweden_df[column_name] = sweden_df[column_name].map(lambda x: int(x))
    sweden_df[column_name] = pd.to_numeric(sweden_df[column_name], errors='coerce')
cumulative_columns_list = [cn for cn in sweden_df.columns if cn.endswith('_Cumulative')]
for column_name in cumulative_columns_list:
    sweden_df[column_name].ffill(inplace=True)
    sweden_df[column_name].fillna(value=0, inplace=True)
    sweden_df[column_name] = sweden_df[column_name].map(lambda x: int(x))
    sweden_df[column_name] = pd.to_numeric(sweden_df[column_name], errors='coerce')
s.store_objects(sweden_df=sweden_df)
columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative']
sweden_df[columns_list].tail(5)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\sweden_df.pickle


,Date,Confirmed_New,Confirmed_Cumulative,Deaths_New,Deaths_Cumulative
56,2020-04-17,688,13746,49,1628
57,2020-04-18,531,14277,51,1679
58,2020-04-19,389,14666,43,1722
59,2020-04-20,462,15128,21,1743
60,2020-04-21,194,15322,3,1765



## United States

In [3]:

from datetime import datetime

tables_url = 'https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population'
tables_list = get_page_tables(tables_url)

[(0, (60, 12)), (2, (71, 10)), (4, (11, 2)), (1, (7, 1)), (3, (2, 2))]


In [4]:

state_populations_df = tables_list[0].copy()
#['{}_{}'.format(column_tuple[0], column_tuple[1]) for column_tuple in state_populations_df.columns.tolist()]
state_populations_df.columns = ['Rank_Current', 'Rank_2010', 'State_Name', 'Census_Estimate_2019', 'Census_Population_2010', 'Change_Percent',
                                'Change_Absolute', 'House_Seats_Total', 'Population_Per_Electoral_Vote_2019', 'Population_Per_House_Seat_2019',
                                'Population_Per_House_Seat_2010', 'Percent_Total_2019']
state_populations_df = state_populations_df.iloc[:-4]
state_populations_df.set_index('State_Name', drop=True, inplace=True)
state_populations_df.Census_Population_2010 = state_populations_df.Census_Population_2010.map(lambda x: re.sub('[^\d]+', '', str(x)))
state_populations_df.Census_Population_2010 = pd.to_numeric(state_populations_df.Census_Population_2010, errors='coerce')
state_populations_df.Census_Population_2010.fillna(value=0, inplace=True)
state_populations_df.Census_Population_2010 = state_populations_df.Census_Population_2010.map(lambda x: int(x))
columns_list = ['Census_Population_2010']
s.store_objects(state_populations_df=state_populations_df)
state_populations_df[columns_list].sample(5)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\state_populations_df.pickle


,Census_Population_2010
State_Name,
Oklahoma,3751351
South Dakota,814180
California,37254523
Illinois,12830632
Florida,18801310


In [116]:

from datetime import datetime

tables_url = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_area'
tables_list = get_page_tables(tables_url)

[(0, (60, 12)), (1, (9, 12)), (2, (4, 12)), (3, (11, 2))]


In [144]:

us_states_area_df = tables_list[0].copy()
us_states_area_df.columns = ['State_Name', 'Total_Area_Rank', 'Total_Area_sq_mi', 'Total_Area_sq_km', 'Land_Area_Rank',
                             'Land_Area_sq_mi', 'Land_Area_sq_km', 'Land_Area_Percent', 'Water_Rank', 'Water_sq_mi',
                             'Water_sq_km', 'Water_Percent']
us_states_area_df = us_states_area_df.iloc[:-3]
for column_name in us_states_area_df.columns:
    if column_name == 'State_Name':
        us_states_area_df[column_name] = us_states_area_df[column_name].map(lambda x: str(x).split('[')[0])
    else:
        us_states_area_df[column_name] = us_states_area_df[column_name].map(lambda x: re.sub('[^\d\.]+', '', str(x)))
        us_states_area_df[column_name] = pd.to_numeric(us_states_area_df[column_name], errors='coerce')
mask_series = (us_states_area_df.State_Name == 'United States Virgin Islands')
us_states_area_df.loc[mask_series, 'State_Name'] = 'U.S. Virgin Islands'
us_states_area_df.set_index('State_Name', drop=True, inplace=True)
state_populations_df = s.load_object('state_populations_df')
state_populations_df = state_populations_df.merge(us_states_area_df, left_index=True, right_index=True)
s.store_objects(state_populations_df=state_populations_df)
state_populations_df.sample(10).T.sample(10)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\state_populations_df.pickle


State_Name,Maine,Guam,Oregon,Virginia,Vermont,Montana,Iowa,Alabama,North Carolina,Wisconsin
Change_Percent,1.2%,4.0%,10.1%,6.7%,-0.3%,8.0%,3.6%,2.6%,10.0%,2.4%
Census_Estimate_2019,1344212,165718,4217737,8535519,623989,1068778,3155070,4903185,10488084,5822434
Population_Per_Electoral_Vote_2019,336053,NaN,602534,656578,207996,356259,525845,544798,699206,582243
Rank_2010,42,53,27,12,50,45,31,23,10,20
Water_sq_mi,4536.82,360.82,2390.53,3284.84,399.71,1493.91,415.68,1774.74,5201.25,11338.6
Total_Area_sq_mi,35379.7,570.62,98378.5,42774.9,9616.36,147040,56272.8,52420.1,53819.2,65496.4
Land_Area_sq_mi,30842.9,209.8,95988,39490.1,9216.66,145546,55857.1,50645.3,48617.9,54157.8
Water_Percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
House_Seats_Total,2,1 (non-voting),5,11,1,1,4,7,13,8
Land_Area_Percent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:

from datetime import datetime

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_the_United_States'
tables_list = get_page_tables(tables_url)

[(6, (58, 8)), (2, (73, 5)), (8, (65, 2)), (5, (42, 2)), (10, (20, 2)), (11, (19, 2)), (1, (12, 2)), (27, (10, 2)), (22, (8, 2)), (23, (7, 2)), (25, (7, 2)), (12, (6, 2)), (16, (6, 2)), (26, (6, 2)), (17, (5, 2)), (15, (4, 2)), (18, (4, 2)), (19, (3, 2)), (20, (3, 2)), (28, (3, 2)), (4, (2, 2)), (14, (2, 2)), (21, (2, 2)), (24, (2, 2)), (29, (2, 2)), (0, (1, 2)), (3, (1, 2)), (7, (1, 2)), (9, (1, 2)), (13, (1, 2))]


In [56]:

states_current_numbers_df = tables_list[4].copy()
states_current_numbers_df.columns = ['d1', 'State_Name', 'Confirmed_Cumulative', 'Deaths_Cumulative', 'Recovered_Cumulative', 'Hospitalized_Cumulative',
                              'd2', 'd3']
columns_list = ['State_Name', 'Confirmed_Cumulative', 'Deaths_Cumulative', 'Recovered_Cumulative', 'Hospitalized_Cumulative']
states_current_numbers_df = states_current_numbers_df[columns_list]
states_current_numbers_df = states_current_numbers_df.iloc[:-2]
states_current_numbers_df.set_index('State_Name', drop=True, inplace=True)
for column_name in states_current_numbers_df.columns:
    states_current_numbers_df[column_name] = states_current_numbers_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
    states_current_numbers_df[column_name] = pd.to_numeric(states_current_numbers_df[column_name], errors='coerce')
    states_current_numbers_df[column_name].fillna(value=0, inplace=True)
    states_current_numbers_df[column_name] = states_current_numbers_df[column_name].map(lambda x: int(x))
s.store_objects(states_current_numbers_df=states_current_numbers_df)
states_current_numbers_df.sample(5)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\states_current_numbers_df.pickle


,Confirmed_Cumulative,Deaths_Cumulative,Recovered_Cumulative,Hospitalized_Cumulative
State_Name,,,,
Colorado,9047,391,0,1755
North Dakota,341,9,138,42
Wyoming,287,2,164,0
Illinois,27575,1134,0,4423
Nebraska,871,18,0,0


In [29]:

lockdowns_df = tables_list[5].copy()
lockdowns_df.columns = ['State_Name', 'Date_Lockdown_Enacted']
date_format = '%B %d, %Y'
lockdowns_df.Date_Lockdown_Enacted = lockdowns_df.Date_Lockdown_Enacted.map(lambda x: datetime.strptime('{}'.format(x), date_format))
lockdowns_df.Date_Lockdown_Enacted = pd.to_datetime(lockdowns_df.Date_Lockdown_Enacted)
lockdowns_df.set_index('State_Name', drop=True, inplace=True)
s.store_objects(states_covid19_lockdowns_df=lockdowns_df)
lockdowns_df.head(5)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\states_covid19_lockdowns_df.pickle


,Date_Lockdown_Enacted
State_Name,
Alabama,2020-04-04
Alaska,2020-03-28
Arizona,2020-03-31
California,2020-03-19
Colorado,2020-03-26


In [5]:

tables_url = 'https://en.wikipedia.org/wiki/Template:2019%E2%80%9320_coronavirus_pandemic_data/United_States_medical_cases'
tables_list = get_page_tables(tables_url)

[(0, (77, 63)), (1, (60, 59)), (2, (66, 2)), (4, (21, 2)), (5, (20, 2)), (17, (8, 2)), (6, (7, 2)), (18, (7, 2)), (20, (7, 2)), (11, (6, 2)), (21, (6, 2)), (12, (5, 2)), (10, (4, 2)), (13, (4, 2)), (14, (3, 2)), (15, (3, 2)), (9, (2, 2)), (16, (2, 2)), (19, (2, 2)), (3, (1, 2)), (7, (1, 2)), (8, (1, 2))]


In [19]:

from datetime import datetime

usa_df = tables_list[0].copy()
usa_df.columns = ['Date', 'West_AK', 'West_AZ', 'West_CA', 'West_CO', 'West_HI', 'West_ID', 'West_MT', 'West_NM', 'West_NV', 'West_OR',
                  'West_UT', 'West_WA', 'West_WY', 'Midwest_IA', 'Midwest_IL', 'Midwest_IN', 'Midwest_KS', 'Midwest_MI', 'Midwest_MN',
                  'Midwest_MO', 'Midwest_ND', 'Midwest_NE', 'Midwest_OH', 'Midwest_OK', 'Midwest_SD', 'Midwest_WI', 'South_AL',
                  'South_AR', 'South_FL', 'South_GA', 'South_KY', 'South_LA', 'South_MS', 'South_NC', 'South_SC', 'South_TN', 'South_TX',
                  'South_VA', 'South_WV', 'Northeast_CT', 'Northeast_DC', 'Northeast_DE', 'Northeast_MA', 'Northeast_MD',
                  'Northeast_ME', 'Northeast_NH', 'Northeast_NJ', 'Northeast_NY', 'Northeast_PA', 'Northeast_RI', 'Northeast_VT',
                  'Territories_GU', 'Territories_MP', 'Territories_PR', 'Territories_VI', 'Date_Date.1', 'Confirmed_New',
                  'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative', 'Recovered_New', 'Recovered_Cumulative']
usa_df = usa_df.iloc[:-5]
columns_list = ['Date', 'West_AK', 'West_AZ', 'West_CA', 'West_CO', 'West_HI', 'West_ID', 'West_MT', 'West_NM', 'West_NV', 'West_OR',
                'West_UT', 'West_WA', 'West_WY', 'Midwest_IA', 'Midwest_IL', 'Midwest_IN', 'Midwest_KS', 'Midwest_MI', 'Midwest_MN',
                'Midwest_MO', 'Midwest_ND', 'Midwest_NE', 'Midwest_OH', 'Midwest_OK', 'Midwest_SD', 'Midwest_WI', 'South_AL',
                'South_AR', 'South_FL', 'South_GA', 'South_KY', 'South_LA', 'South_MS', 'South_NC', 'South_SC', 'South_TN', 'South_TX',
                'South_VA', 'South_WV', 'Northeast_CT', 'Northeast_DC', 'Northeast_DE', 'Northeast_MA', 'Northeast_MD',
                'Northeast_ME', 'Northeast_NH', 'Northeast_NJ', 'Northeast_NY', 'Northeast_PA', 'Northeast_RI', 'Northeast_VT',
                'Territories_GU', 'Territories_MP', 'Territories_PR', 'Territories_VI', 'Confirmed_New',
                'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative', 'Recovered_New', 'Recovered_Cumulative']
usa_df = usa_df[columns_list]
idx_list = list(range(16)) + list(range(18, 49)) + list(range(51, 72))
mask_series = usa_df.index.isin(idx_list)
usa_df = usa_df[mask_series]
date_format = '%b %d, %Y'
for column_name in usa_df.columns:
    if column_name == 'Date':
        usa_df[column_name] = usa_df[column_name].map(lambda x: datetime.strptime('{}, 2020'.format(x), date_format))
        usa_df[column_name] = pd.to_datetime(usa_df[column_name])
    else:
        usa_df[column_name] = usa_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        usa_df[column_name] = pd.to_numeric(usa_df[column_name], errors='coerce')
for column_name in [cn for cn in usa_df.columns if cn.endswith('_Cumulative')]:
    usa_df[column_name].ffill(inplace=True)
for column_name in [cn for cn in usa_df.columns if cn.endswith('_New') or cn.endswith('_Cumulative')]:
    usa_df[column_name].fillna(value=0, inplace=True)
    usa_df[column_name] = usa_df[column_name].map(lambda x: int(x))
    usa_df[column_name] = pd.to_numeric(usa_df[column_name], errors='coerce')
s.store_objects(usa_df=usa_df)
columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative']
usa_df.tail(5)[columns_list]

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\usa_df.pickle


,Date,Confirmed_New,Confirmed_Cumulative,Deaths_New,Deaths_Cumulative
67,2020-04-17,30915,696621,2080,32435
68,2020-04-18,28084,724705,1743,34178
69,2020-04-19,26013,750718,1634,35812
70,2020-04-20,25093,775811,1643,37455
71,2020-04-21,25311,801122,2627,40082


In [ ]:

us_deaths_df = tables_list[1].copy()
us_deaths_df.columns = ['Date', 'West_AK', 'West_AZ', 'West_CA', 'West_CO', 'West_HI', 'West_ID', 'West_MT', 'West_NM', 'West_NV',
                        'West_OR', 'West_UT', 'West_WA', 'West_WY', 'Midwest_IA', 'Midwest_IL', 'Midwest_IN', 'Midwest_KS',
                        'Midwest_MI', 'Midwest_MN', 'Midwest_MO', 'Midwest_ND', 'Midwest_NE', 'Midwest_OH', 'Midwest_OK', 'Midwest_SD',
                        'Midwest_WI', 'South_AL', 'South_AR', 'South_FL', 'South_GA', 'South_KY', 'South_LA', 'South_MS', 'South_NC',
                        'South_SC', 'South_TN', 'South_TX', 'South_VA', 'South_WV', 'Northeast_CT', 'Northeast_DC', 'Northeast_DE',
                        'Northeast_MA', 'Northeast_MD', 'Northeast_ME', 'Northeast_NH', 'Northeast_NJ', 'Northeast_NY',
                        'Northeast_PA', 'Northeast_RI', 'Northeast_VT', 'Territories_GU', 'Territories_MP', 'Territories_PR', 'Territories_VI',
                        'Deaths_New', 'Deaths_Cumulative']
us_deaths_df = us_deaths_df.iloc[:-5]
date_format = '%b %d, %Y'
for column_name in us_deaths_df.columns:
    if column_name == 'Date':
        us_deaths_df[column_name] = us_deaths_df[column_name].map(lambda x: datetime.strptime('{}, 2020'.format(x), date_format))
        us_deaths_df[column_name] = pd.to_datetime(us_deaths_df[column_name])
        pass
    else:
        us_deaths_df[column_name] = us_deaths_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        us_deaths_df[column_name] = pd.to_numeric(us_deaths_df[column_name], errors='coerce')
#us_deaths_df.set_index('Date', drop=True, inplace=True)
s.store_objects(us_deaths_df=us_deaths_df)
#columns_list = ['Date', 'Deaths_New', 'Deaths_Cumulative']
#us_deaths_df[columns_list]


----
## Italy

In [ ]:

#tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Italy'
tables_url = 'https://en.wikipedia.org/wiki/Template:2019%E2%80%9320_coronavirus_pandemic_data/Italy_medical_cases'
tables_list = get_page_tables(tables_url)

In [69]:

xlsx_dir = os.path.join(s.data_folder, 'xlsx')
file_path = os.path.join(xlsx_dir, 'italy.xlsx')
italy_df = pd.read_excel(file_path)
italy_df.columns = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative', 'Active_ICU', 'Active_Cumulative',
                    'Recovered_Cumulative', 'Tested_Cumulative']
for column_name in italy_df.columns:
    if column_name == 'Date':
        italy_df[column_name] = pd.to_datetime(italy_df[column_name])
        pass
    else:
        italy_df[column_name] = italy_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        italy_df[column_name] = pd.to_numeric(italy_df[column_name], errors='coerce')
s.store_objects(italy_df=italy_df)
columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative']
italy_df[columns_list].tail(5)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\italy_df.pickle


,Date,Confirmed_New,Confirmed_Cumulative,Deaths_New,Deaths_Cumulative
53,2020-04-12,4092,156363,431.0,19899.0
54,2020-04-13,3153,159516,566.0,20465.0
55,2020-04-14,2972,162488,602.0,21067.0
56,2020-04-15,2667,165155,578.0,21645.0
57,2020-04-16,3786,168941,525.0,22170.0


In [ ]:

italy_df = s.load_object('italy_df')
italy_df.set_index('Date', drop=True, inplace=True)
columns_list = ['Deaths_New', 'Deaths_Cumulative']
tracking_df = us_deaths_df[columns_list].merge(italy_df[columns_list], how='outer',
                                               left_index=True, right_index=True, suffixes=('_usa', '_italy'))
for column_name in tracking_df.columns:
    tracking_df[column_name] = pd.to_numeric(tracking_df[column_name], errors='coerce')


---
## Spain

In [62]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Spain'
tables_list = get_page_tables(tables_url)

[(6, (64, 30)), (7, (61, 30)), (2, (62, 5)), (15, (65, 2)), (5, (21, 6)), (9, (12, 10)), (10, (12, 10)), (11, (12, 10)), (17, (20, 2)), (18, (19, 2)), (1, (11, 2)), (12, (6, 3)), (13, (6, 3)), (29, (8, 2)), (30, (7, 2)), (32, (7, 2)), (19, (6, 2)), (23, (6, 2)), (33, (6, 2)), (24, (5, 2)), (22, (4, 2)), (25, (4, 2)), (26, (3, 2)), (27, (3, 2)), (21, (2, 2)), (28, (2, 2)), (31, (2, 2)), (0, (1, 2)), (3, (1, 2)), (8, (1, 2)), (14, (1, 2)), (16, (1, 2)), (20, (1, 2)), (4, (1, 1))]


In [63]:

spain_df = tables_list[6].copy()
districts_list = ['AN', 'AR', 'AS', 'IB', 'CN', 'CB', 'CM', 'CL', 'CT', 'CE', 'VC', 'EX', 'GA', 'MD', 'ML', 'MU', 'NA',
                  'PV', 'RI']
summaries_list = ['Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative', 'ICU_Cumulative',
                  'Recovered_Cumulative', 'Tested_Cumulative']
spain_df.columns = ['Date', 'Time'] + districts_list + summaries_list + ['Refs', 'Notes']
spain_df = spain_df.loc[3:]
spain_df = spain_df.iloc[:-6]
for column_name in spain_df.columns:
    if column_name == 'Date':
        spain_df[column_name] = pd.to_datetime(spain_df[column_name])
        pass
    elif column_name == 'Time':
        spain_df[column_name] = pd.to_timedelta(spain_df[column_name]+':00')
        pass
    else:
        spain_df[column_name] = spain_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        spain_df[column_name] = pd.to_numeric(spain_df[column_name], errors='coerce')
spain_df.reset_index(level=0, inplace=True)
s.store_objects(spain_df=spain_df)
columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative']
spain_df.tail(5)[columns_list]

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\spain_df.pickle


,Date,Confirmed_New,Confirmed_Cumulative,Deaths_New,Deaths_Cumulative
50,2020-04-12,3477,169496.0,517.0,17489.0
51,2020-04-13,3045,172541.0,567.0,18056.0
52,2020-04-14,5092,177633.0,523.0,18579.0
53,2020-04-15,5183,182816.0,551.0,19130.0
54,2020-04-16,5252,188068.0,348.0,19478.0


In [ ]:

tables_url = 'https://en.wikipedia.org/wiki/Template:2019%E2%80%9320_coronavirus_pandemic_data/Spain_medical_cases'
tables_list = get_page_tables(tables_url)

In [ ]:

df_columns_list = districts_list + summaries_list
df = pd.DataFrame([], columns=df_columns_list)
summed_list = districts_list + ['Confirmed_New', 'Deaths_New']
for column_name in summed_list:
    new_group = spain_df.groupby('Date')[column_name]
    df[column_name] = new_group.agg(np.sum)
for column_name in ['Confirmed_Cumulative', 'Deaths_Cumulative', 'ICU_Cumulative', 'Recovered_Cumulative', 'Tested_Cumulative']:
    cumulative_group = spain_df.groupby('Date')[column_name]
    df[column_name] = cumulative_group.agg(max)


----
## France

In [98]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_France'
tables_list = get_page_tables(tables_url)

[(9, (55, 25)), (8, (105, 7)), (3, (54, 5)), (11, (65, 2)), (4, (6, 10)), (13, (20, 2)), (14, (19, 2)), (2, (13, 2)), (25, (8, 2)), (26, (7, 2)), (28, (7, 2)), (15, (6, 2)), (19, (6, 2)), (29, (6, 2)), (20, (5, 2)), (30, (5, 2)), (18, (4, 2)), (21, (4, 2)), (22, (3, 2)), (23, (3, 2)), (17, (2, 2)), (24, (2, 2)), (27, (2, 2)), (0, (1, 2)), (1, (1, 2)), (5, (1, 2)), (6, (1, 2)), (7, (1, 2)), (10, (1, 2)), (12, (1, 2)), (16, (1, 2))]


In [83]:

france_df = tables_list[9].copy()
france_df.columns = ['Date', 'Metropolitan_ARA', 'Metropolitan_BFC', 'Metropolitan_BRE', 'Metropolitan_CVL', 'Metropolitan_COR',
                     'Metropolitan_GES', 'Metropolitan_HDF', 'Metropolitan_IDF', 'Metropolitan_NOR', 'Metropolitan_NAQ', 'Metropolitan_OCC',
                     'Metropolitan_PDL', 'Metropolitan_PACA', 'Overseas_La_R', 'Overseas_S_M', 'Overseas_S_B',
                     'Overseas_Mart', 'Overseas_Guad', 'Overseas_May', 'Overseas_Guy', 'Confirmed_New', 'Confirmed_Cumulative',
                     'Deaths_New', 'Deaths_Cumulative']
france_df = france_df.iloc[:-4]
for column_name in france_df.columns:
    if column_name == 'Date':
        france_df[column_name] = pd.to_datetime(france_df[column_name])
    else:
        france_df[column_name] = france_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        france_df[column_name] = pd.to_numeric(france_df[column_name], errors='coerce')
france_df.Deaths_Cumulative = france_df.Deaths_Cumulative.ffill()
france_df.Deaths_New.fillna(value=0, inplace=True)
france_df.Confirmed_New.fillna(value=0, inplace=True)
s.store_objects(france_df=france_df)
columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative']
france_df[columns_list].tail(5)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\france_df.pickle


,Date,Confirmed_New,Confirmed_Cumulative,Deaths_New,Deaths_Cumulative
46,2020-04-04,2572.0,54097,0.0,2.0
47,2020-04-05,1846.0,55943,0.0,2.0
48,2020-04-06,2494.0,58437,0.0,2.0
49,2020-04-07,2986.0,61423,0.0,2.0
50,2020-04-08,2804.0,64227,0.0,2.0



---
## United Kingdom

In [85]:

#tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_the_United_Kingdom'
tables_url = 'https://en.wikipedia.org/wiki/Template:2019%E2%80%9320_coronavirus_pandemic_data/United_Kingdom_medical_cases'
tables_list = get_page_tables(tables_url)

[(0, (81, 21)), (1, (65, 2)), (3, (20, 2)), (4, (19, 2)), (15, (8, 2)), (16, (7, 2)), (18, (7, 2)), (5, (6, 2)), (9, (6, 2)), (19, (6, 2)), (10, (5, 2)), (8, (4, 2)), (11, (4, 2)), (12, (3, 2)), (13, (3, 2)), (7, (2, 2)), (14, (2, 2)), (17, (2, 2)), (2, (1, 2)), (6, (1, 2))]


In [95]:

uk_df = tables_list[0].copy()
#['{}_{}'.format(column_tuple[0], column_tuple[1]) for column_tuple in uk_df.columns.tolist()]
uk_df.columns = ['Date', 'England_East', 'England_London', 'England_Midlands', 'England_NE_Yorks', 'England_North_West',
                 'England_South_East', 'England_South_West', 'England_Unclassified', 'Northern_Ireland', 'Scotland', 'Wales',
                 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_Hospital_New', 'Deaths_Hospital_Cumulative',
                 'Deaths_Certificate_New', 'Deaths_Certificate_Cumulative', 'Tested_New', 'Tested_Cumulative', 'Sources']
uk_df = uk_df.iloc[:-4]
date_format = '%d %b, %Y'
for column_name in uk_df.columns:
    if column_name == 'Date':
        uk_df[column_name] = uk_df[column_name].map(lambda x: datetime.strptime('{}, 2020'.format(re.sub('\\xa0', ' ', x)), date_format))
        uk_df[column_name] = pd.to_datetime(uk_df[column_name])
        pass
    else:
        uk_df[column_name] = uk_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        uk_df[column_name] = pd.to_numeric(uk_df[column_name], errors='coerce')
for column_name in [cn for cn in uk_df.columns if cn.endswith('_New')]:
    uk_df[column_name].fillna(value=0, inplace=True)
for column_name in [cn for cn in uk_df.columns if cn.endswith('_Cumulative')]:
    uk_df[column_name].ffill(inplace=True)
    uk_df[column_name].fillna(value=0, inplace=True)
s.store_objects(uk_df=uk_df)
columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_Hospital_New', 'Deaths_Hospital_Cumulative',
                 'Deaths_Certificate_New', 'Deaths_Certificate_Cumulative']
uk_df[columns_list].tail(5)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\uk_df.pickle


,Date,Confirmed_New,Confirmed_Cumulative,Deaths_Hospital_New,Deaths_Hospital_Cumulative,Deaths_Certificate_New,Deaths_Certificate_Cumulative
72,2020-04-12,5288.0,84279,717.0,11329.0,0.0,6235.0
73,2020-04-13,4342.0,88621,778.0,12107.0,0.0,6235.0
74,2020-04-14,5252.0,93873,761.0,12868.0,0.0,6235.0
75,2020-04-15,4605.0,98476,861.0,13729.0,0.0,6235.0
76,2020-04-16,4618.0,103093,0.0,13729.0,0.0,6235.0



---
## Philippines

In [15]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_the_Philippines'
tables_list = get_page_tables(tables_url)

[(3, (217, 10)), (2, (54, 5)), (6, (20, 10)), (14, (65, 2)), (7, (18, 7)), (8, (15, 7)), (16, (20, 2)), (17, (19, 2)), (36, (12, 3)), (4, (7, 5)), (1, (12, 2)), (11, (12, 2)), (33, (10, 2)), (28, (8, 2)), (10, (5, 3)), (29, (7, 2)), (31, (7, 2)), (18, (6, 2)), (22, (6, 2)), (32, (6, 2)), (23, (5, 2)), (37, (5, 2)), (21, (4, 2)), (24, (4, 2)), (25, (3, 2)), (26, (3, 2)), (20, (2, 2)), (27, (2, 2)), (30, (2, 2)), (34, (2, 2)), (35, (2, 2)), (0, (1, 2)), (5, (1, 2)), (9, (1, 2)), (12, (1, 2)), (13, (1, 2)), (15, (1, 2)), (19, (1, 2))]


In [23]:

philippines_df = tables_list[2].copy()
philippines_df.columns = philippines_df.iloc[0].tolist()
columns_list = [1, 3]
philippines_df = philippines_df.iloc[:, columns_list]
philippines_df.columns = ['Date', 'Confirmed_New']
#philippines_df.Date.to_dict()
idx_list = [1] + [3] + [5] + [7] + [9] + list(range(11, 53))
mask_series = philippines_df.index.isin(idx_list)
philippines_df = philippines_df[mask_series]
for column_name in philippines_df.columns:
    if column_name == 'Date':
        philippines_df[column_name] = pd.to_datetime(philippines_df[column_name])
    else:
        philippines_df[column_name] = philippines_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        philippines_df[column_name] = pd.to_numeric(philippines_df[column_name], errors='coerce')
s.store_objects(philippines_df=philippines_df)
philippines_df.tail(5)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\philippines_df.pickle


,Date,Confirmed_New
48,2020-04-13,4932
49,2020-04-14,5223
50,2020-04-15,5453
51,2020-04-16,5660
52,2020-04-17,5878



---
## Thailand

In [52]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Thailand'
tables_list = get_page_tables(tables_url)

[(5, (342, 12)), (3, (65, 5)), (6, (19, 7)), (7, (65, 2)), (9, (22, 2)), (10, (21, 2)), (1, (13, 2)), (19, (10, 2)), (20, (9, 2)), (22, (8, 2)), (23, (7, 2)), (4, (2, 6)), (11, (5, 2)), (12, (5, 2)), (15, (4, 2)), (21, (4, 2)), (13, (3, 2)), (16, (3, 2)), (17, (3, 2)), (18, (2, 2)), (0, (1, 2)), (2, (1, 2)), (8, (1, 2)), (14, (1, 2))]


In [59]:

thailand_df = tables_list[3].copy()
thailand_df.columns = thailand_df.iloc[0].tolist()
columns_list = [1, 3]
thailand_df = thailand_df.iloc[:, columns_list]
thailand_df.columns = ['Date', 'Confirmed_New']
#thailand_df.Date.to_dict()
idx_list = [1, 3,] + list(range(5, 20)) + list(range(21, 26)) + list(range(27, 29)) + list(range(30, 42)) + list(range(43, 64))
mask_series = thailand_df.index.isin(idx_list)
thailand_df = thailand_df[mask_series]
for column_name in thailand_df.columns:
    if column_name == 'Date':
        thailand_df[column_name] = pd.to_datetime(thailand_df[column_name])
        pass
    else:
        thailand_df[column_name] = thailand_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        thailand_df[column_name] = pd.to_numeric(thailand_df[column_name], errors='coerce')
s.store_objects(thailand_df=thailand_df)

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\thailand_df.pickle



----
## Vietnam

In [41]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Vietnam'
tables_list = get_page_tables(tables_url)

[(4, (121, 12)), (2, (47, 5)), (5, (65, 2)), (3, (21, 3)), (7, (22, 2)), (8, (21, 2)), (1, (11, 2)), (17, (10, 2)), (18, (9, 2)), (20, (8, 2)), (21, (7, 2)), (9, (5, 2)), (10, (5, 2)), (13, (4, 2)), (19, (4, 2)), (11, (3, 2)), (14, (3, 2)), (15, (3, 2)), (16, (2, 2)), (0, (1, 2)), (6, (1, 2)), (12, (1, 2))]


In [51]:

vietnam_df = tables_list[2].copy()
vietnam_df.columns = vietnam_df.iloc[0].tolist()
columns_list = [1, 3]
vietnam_df = vietnam_df.iloc[:, columns_list]
vietnam_df.columns = ['Date', 'Confirmed_New']
#vietnam_df.Date.to_dict()
idx_list = [1, 3,] + list(range(5, 19)) + list(range(20, 23)) + [24] + list(range(26, 46))
mask_series = vietnam_df.index.isin(idx_list)
vietnam_df = vietnam_df[mask_series]
for column_name in vietnam_df.columns:
    if column_name == 'Date':
        vietnam_df[column_name] = pd.to_datetime(vietnam_df[column_name])
        pass
    else:
        vietnam_df[column_name] = vietnam_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        vietnam_df[column_name] = pd.to_numeric(vietnam_df[column_name], errors='coerce')
s.store_objects(vietnam_df=vietnam_df)

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\vietnam_df.pickle



---
## Malaysia

In [29]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Malaysia'
tables_list = get_page_tables(tables_url)

[(8, (120, 8)), (6, (113, 5)), (7, (46, 7)), (3, (56, 5)), (5, (13, 18)), (9, (20, 9)), (12, (65, 2)), (14, (22, 2)), (15, (21, 2)), (2, (16, 2)), (4, (6, 5)), (24, (10, 2)), (25, (9, 2)), (11, (4, 4)), (27, (8, 2)), (10, (3, 5)), (28, (7, 2)), (16, (5, 2)), (17, (5, 2)), (20, (4, 2)), (26, (4, 2)), (18, (3, 2)), (21, (3, 2)), (22, (3, 2)), (23, (2, 2)), (0, (1, 2)), (1, (1, 2)), (13, (1, 2)), (19, (1, 2))]


In [40]:

from datetime import datetime

malaysia_df = tables_list[7].copy()
malaysia_df.columns = ['Date', 'Selangor', 'Johor', 'Kuala_Lumpur', 'Kedah', 'North_Sembilan', 'Confirmed_New']
malaysia_df = malaysia_df.loc[:43]
date_format = '%d %B, %Y'
for column_name in malaysia_df.columns:
    if column_name == 'Date':
        malaysia_df[column_name] = malaysia_df[column_name].map(lambda x: datetime.strptime('{}, 2020'.format(x), date_format))
        malaysia_df[column_name] = pd.to_datetime(malaysia_df[column_name])
        pass
    else:
        malaysia_df[column_name] = malaysia_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        malaysia_df[column_name] = pd.to_numeric(malaysia_df[column_name], errors='coerce')
s.store_objects(malaysia_df=malaysia_df)
#columns_list = ['Date', 'Confirmed_New']
#malaysia_df[columns_list]

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\malaysia_df.pickle



----
## South Korea

In [3]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_South_Korea'
tables_list = get_page_tables(tables_url)

[(8, (75, 28)), (2, (30, 13)), (3, (61, 5)), (5, (20, 7)), (7, (20, 7)), (9, (62, 2)), (4, (13, 7)), (6, (8, 5)), (11, (19, 2)), (12, (18, 2)), (1, (14, 2)), (21, (10, 2)), (22, (9, 2)), (24, (8, 2)), (25, (7, 2)), (14, (5, 2)), (17, (4, 2)), (23, (4, 2)), (15, (3, 2)), (18, (3, 2)), (19, (3, 2)), (13, (2, 2)), (20, (2, 2)), (0, (1, 2)), (10, (1, 2)), (16, (1, 2))]


In [6]:

import numpy as np

south_korea_df = tables_list[8].copy()
districts_list = ['Gyeonggi_Incheon', 'Gyeonggi_Seoul', 'Gyeonggi_Gyeonggi', 'Gangwon_Gangwon',
                  'Gyeongsang_Gyeongbuk', 'Gyeongsang_Daegu', 'Gyeongsang_Gyeongnam', 'Gyeongsang_Busan',
                  'Gyeongsang_Ulsan', 'Chungcheong_Chungbuk', 'Chungcheong_Sejong', 'Chungcheong_Daejeon',
                  'Chungcheong_Chungnam', 'Jeolla_Jeonbuk', 'Jeolla_Gwangju', 'Jeolla_Jeonnam',
                  'Jeolla_Jeju', 'Airport Screening']
summaries_list = ['Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative',
                  'Tested_Cumulative', 'Tested_New', 'Discharged_Cumulative']
south_korea_df.columns = ['Date', 'Time'] + districts_list + summaries_list + ['Sources']
#south_korea_df.Date.to_dict()
idx_list = list(range(18)) + list(range(19, 42)) + list(range(43, 69))
mask_series = south_korea_df.index.isin(idx_list)
south_korea_df = south_korea_df[mask_series]
for column_name in south_korea_df.columns:
    if column_name == 'Date':
        south_korea_df[column_name] = pd.to_datetime(south_korea_df[column_name])
        pass
    elif column_name == 'Time':
        south_korea_df[column_name] = pd.to_timedelta(south_korea_df[column_name]+':00')
        pass
    else:
        south_korea_df[column_name] = south_korea_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        south_korea_df[column_name] = pd.to_numeric(south_korea_df[column_name], errors='coerce')
df_columns_list = districts_list + summaries_list
df = pd.DataFrame([], columns=df_columns_list)
summed_list = districts_list + ['Confirmed_New', 'Deaths_New', 'Tested_New']
for column_name in summed_list:
    new_group = south_korea_df.groupby('Date')[column_name]
    df[column_name] = new_group.agg(np.sum)
for column_name in ['Confirmed_Cumulative', 'Deaths_Cumulative', 'Tested_Cumulative', 'Discharged_Cumulative']:
    cumulative_group = south_korea_df.groupby('Date')[column_name]
    df[column_name] = cumulative_group.agg(max)
df.reset_index(level=0, inplace=True)
s.store_objects(south_korea_df=df)
#columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative']
#df[columns_list]

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\south_korea_df.pickle



----
## Japan

In [172]:

tables_url = 'https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_Japan'
tables_list = get_page_tables(tables_url)

[(4, (62, 50)), (3, (55, 7)), (2, (70, 5)), (5, (62, 2)), (7, (19, 2)), (8, (18, 2)), (1, (10, 2)), (17, (10, 2)), (18, (9, 2)), (20, (8, 2)), (21, (7, 2)), (10, (5, 2)), (13, (4, 2)), (19, (4, 2)), (11, (3, 2)), (14, (3, 2)), (15, (3, 2)), (9, (2, 2)), (16, (2, 2)), (0, (1, 2)), (6, (1, 2)), (12, (1, 2))]


In [179]:

japan_df = tables_list[4].copy()
japan_df.columns = ['Date', 'Hokkaidō', 'Honshū_Tōhoku_Aomori', 'Honshū_Tōhoku_Akita',
                    'Honshū_Tōhoku_Miyagi', 'Honshū_Tōhoku_Fukushima', 'Honshū_Kantō_Tochigi', 'Honshū_Kantō_Ibaraki',
                    'Honshū_Kantō_Chiba', 'Honshū_Kantō_Tōkyō', 'Honshū_Kantō_Kanagawa', 'Honshū_Kantō_Saitama',
                    'Honshū_Kantō_Gunma', 'Honshū_Chūbu_Niigata', 'Honshū_Chūbu_Nagano', 'Honshū_Chūbu_Yamanashi',
                    'Honshū_Chūbu_Shizuoka', 'Honshū_Chūbu_Aichi', 'Honshū_Chūbu_Ishikawa', 'Honshū_Chūbu_Fukui',
                    'Honshū_Chūbu_Gifu', 'Honshū_Kansai_Mie', 'Honshū_Kansai_Shiga', 'Honshū_Kansai_Wakayama', 'Honshū_Kansai_Nara',
                    'Honshū_Kansai_Kyōto', 'Honshū_Kansai_Ōsaka', 'Honshū_Kansai_Hyōgo', 'Honshū_Chūgoku_Okayama',
                    'Honshū_Chūgoku_Hiroshima', 'Honshū_Chūgoku_Yamaguchi', 'Shikoku_Shikoku_Kagawa',
                    'Shikoku_Shikoku_Tokushima', 'Shikoku_Shikoku_Kōchi', 'Shikoku_Shikoku_Ehime', 'Kyūshū_Kyūshū_Ōita',
                    'Kyūshū_Kyūshū_Fukuoka', 'Kyūshū_Kyūshū_Saga', 'Kyūshū_Kyūshū_Nagasaki', 'Kyūshū_Kyūshū_Kumamoto',
                    'Kyūshū_Kyūshū_Miyazaki', 'Okinawa', 'Airport', 'Diamond_Princess_Workers', 'Repatriated_Total',
                    'Abroad_Total', 'Confirmed_New', 'Confirmed_Total', 'Tested_Total', 'Sources']
#japan_df.Date.to_dict()
idx_list = list(range(14)) + list(range(15, 32)) + list(range(33, 56))
mask_series = japan_df.index.isin(idx_list)
japan_df = japan_df[mask_series]
#date_format = '%Y/%M/%d'
for column_name in japan_df.columns:
    if column_name == 'Date':
        #japan_df[column_name] = japan_df[column_name].map(lambda x: datetime.strptime('{}'.format(x), date_format))
        japan_df[column_name] = pd.to_datetime(japan_df[column_name])
        pass
    else:
        japan_df[column_name] = japan_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        japan_df[column_name] = pd.to_numeric(japan_df[column_name], errors='coerce')
s.store_objects(japan_df=japan_df)
#columns_list = ['Date', 'Confirmed_Total', 'Confirmed_New']
#japan_df[columns_list]

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\japan_df.pickle



----
## Slovakia

In [34]:

slovakia_df = s.load_object('slovakia_df')
slovakia_df.columns = ['Date', 'Bratislava', 'Žilina', 'Košice', 'Trnava', 'Trenčín', 'Prešov', 'Banská_Bystrica',
                       'Nitra', 'Confirmed_New', 'Confirmed_Total', 'Deaths_New', 'Deaths_Total',
                       'Recoveries_Total', 'Tested_Total', 'References']
slovakia_df = slovakia_df.loc[:17]
for column_name in slovakia_df.columns:
    if column_name == 'Date':
        slovakia_df[column_name] = pd.to_datetime(slovakia_df[column_name])
        pass
    else:
        slovakia_df[column_name] = slovakia_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        slovakia_df[column_name] = pd.to_numeric(slovakia_df[column_name], errors='coerce')
s.store_objects(slovakia_df=slovakia_df)
#columns_list = ['Date', 'Confirmed_New', 'Confirmed_Total', 'Deaths_New', 'Deaths_Total']
#slovakia_df[columns_list]

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\slovakia_df.pickle


In [28]:

singapore_df = s.load_object('singapore_df')
singapore_df.columns = singapore_df.iloc[0].tolist()
singapore_df = singapore_df.loc[1:61]
columns_list = [1, 3, 4]
singapore_df = singapore_df.iloc[:, columns_list]
singapore_df.columns = ['Date', 'Confirmed_New', 'Deaths_New']
idx_list = list(range(11)) + list(range(12, 62))
mask_series = singapore_df.index.isin(idx_list)
singapore_df = singapore_df[mask_series]
for column_name in singapore_df.columns:
    if column_name == 'Date':
        singapore_df[column_name] = pd.to_datetime(singapore_df[column_name])
        pass
    else:
        singapore_df[column_name] = singapore_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        singapore_df[column_name] = pd.to_numeric(singapore_df[column_name], errors='coerce')
s.store_objects(singapore_df=singapore_df)

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\singapore_df.pickle


In [19]:

netherlands_df = s.load_object('netherlands_df')
netherlands_df.columns = netherlands_df.iloc[0].tolist()
netherlands_df = netherlands_df.loc[1:27]
columns_list = [1, 3, 4]
netherlands_df = netherlands_df.iloc[:, columns_list]
netherlands_df.columns = ['Date', 'Confirmed_New', 'Deaths_New']
for column_name in netherlands_df.columns:
    if column_name == 'Date':
        netherlands_df[column_name] = pd.to_datetime(netherlands_df[column_name])
        pass
    else:
        netherlands_df[column_name] = netherlands_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
        netherlands_df[column_name] = pd.to_numeric(netherlands_df[column_name], errors='coerce')
s.store_objects(netherlands_df=netherlands_df)

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\netherlands_df.pickle


In [ ]:

hong_kong_df = s.load_object('hong_kong_df')
hong_kong_df.columns = hong_kong_df.iloc[0].tolist()
hong_kong_df = hong_kong_df.loc[1:62]
columns_list = [1, 4]
hong_kong_df = hong_kong_df.iloc[:, columns_list]
hong_kong_df.columns = ['Date', 'Confirmed_New']
hong_kong_df.Confirmed_New = hong_kong_df.Confirmed_New.map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
hong_kong_df.Confirmed_New = pd.to_numeric(hong_kong_df.Confirmed_New, errors='coerce')
hong_kong_df.Date = pd.to_datetime(hong_kong_df.Date)
s.store_objects(hong_kong_df=hong_kong_df)

In [ ]:

germany_df = s.load_object('germany_df')
germany_df.columns = ['Date', 'Baden-Württemberg', 'Bavaria', 'Berlin', 'Brandenburg', 'Bremen', 'Hamburg', 'Hesse',
                      'Mecklenburg-Vorpommern', 'Lower Saxony', 'North Rhine-Westphalia', 'Rhineland-Palatinate', 'Saarland',
                      'Saxony', 'Saxony-Anhalt', 'Schleswig-Holstein', 'Thuringia', 'Confirmed_Repatriated', 'Confirmed_Total',
                      'Deaths_Total', 'Confirmed_New', 'Deaths_New']
date_format = '%d %b, %Y'
for column_name in germany_df.columns:
    if column_name == 'Date':
        germany_df[column_name] = germany_df[column_name].map(lambda x: datetime.strptime('{}, 2020'.format(x), date_format))
        germany_df[column_name] = pd.to_datetime(germany_df[column_name])
        pass
    else:
        germany_df[column_name] = germany_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        germany_df[column_name] = pd.to_numeric(germany_df[column_name], errors='coerce')
s.store_objects(germany_df=germany_df)
#columns_list = ['Date', 'Confirmed_Repatriated', 'Confirmed_Total', 'Deaths_Total', 'Confirmed_New', 'Deaths_New']
#germany_df[columns_list]

In [ ]:

czech_df = s.load_object('czech_df')
czech_df.columns = ['Date', 'Confirmed_New', 'Confirmed_Total', 'Recovered_New', 'Recovered_Total', 'Deaths_New', 'Deaths_Total',
                    'Tested_Total']
date_format = '%d %B, %Y'
for column_name in czech_df.columns:
    if column_name == 'Date':
        czech_df[column_name] = czech_df[column_name].map(lambda x: datetime.strptime('{}, 2020'.format(x), date_format))
        czech_df[column_name] = pd.to_datetime(czech_df[column_name])
        pass
    else:
        czech_df[column_name] = czech_df[column_name].map(lambda x: re.sub('[^\d]+', '', str(x)))
        czech_df[column_name] = pd.to_numeric(czech_df[column_name], errors='coerce')
s.store_objects(czech_df=czech_df)

In [ ]:

china_df = s.load_object('china_df')
china_df.columns = china_df.iloc[0].tolist()
china_df = china_df.loc[1:77]
columns_list = [1, 4]
china_df = china_df.iloc[:, columns_list]
china_df.columns = ['Date', 'Confirmed_New']
china_df.Confirmed_New = china_df.Confirmed_New.map(lambda x: re.sub('[^\d]+', '', str(x).split('(')[0]))
china_df.Confirmed_New = pd.to_numeric(china_df.Confirmed_New, errors='coerce')
china_df.Date = pd.to_datetime(china_df.Date)
s.store_objects(china_df=china_df)

In [ ]:

page_title = '2019–20 coronavirus pandemic'
page_obj = wikipedia.page(title=page_title)
print(['page_obj.{}'.format(fn) for fn in dir(page_obj) if 'link' in fn.lower()])

In [ ]:

print(['page_obj.{}'.format(fn) for fn in dir(page_obj) if 'url' in fn.lower()])

In [ ]:

countries_list = ['Italy']
locations_list = [link for link in set(page_obj.links) if ' in ' in link]
for page_title in locations_list:
    if any(map(lambda x: x in page_title, countries_list)):
        page_obj = wikipedia.page(title=page_title)
        tables_url = page_obj.url
        tables_list = get_page_tables(tables_url)
        break

In [ ]:

print(page_title)
tables_list[4]

In [ ]:

locations_list

In [ ]:

pandemic_df = s.load_object('pandemic_df')
pandemic_df.loc[12, 'R0_low'] = 2
pandemic_df.loc[12, 'R0_high'] = 6
s.store_objects(pandemic_df=pandemic_df)

In [ ]:

pandemic_df.set_index('Disease', drop=True, inplace=True)
s.store_objects(pandemic_df=pandemic_df)

In [ ]:

csv_name = 'COVID-19'
csv_path = os.path.abspath(os.path.join(s.data_csv_folder, '{}.csv'.format(csv_name)))
covid19_df = pd.read_csv(csv_path, encoding=s.encoding_type, header=[0, 1])
covid19_df

In [ ]:

name_regex = re.compile(r'[^0-9A-Za-z]+')
[name_regex.sub('_', '{} {}'.format(cn_tuple[0], cn_tuple[1]).lower()) for cn_tuple in covid19_df.columns]

In [ ]:

covid19_df.columns = ['Disease', 'deaths_per_day_global_rounded', 'total_news_mentions_millions_data_retrieved_3rd_mar_2020',
                      'news_mentions_per_death', 'CFR_low', 'R0_low', 'annual_global_fatalities_latest_data_year',
                      'new_cases_per_year_latest_data_year', 'days_of_outbreak', 'fatalty_notes', 'source', 'url']
covid19_df.set_index('Disease', drop=True, inplace=True)
s.store_objects(covid19_df=covid19_df)

In [ ]:

pandemic_df = s.load_object('pandemic_df')
pandemic_df.index = ['Pertussis', 'Diphtheria', 'Measles', 'Smallpox', 'Rubella', 'Mumps', 'SARS', 'MERS', 'Swine flu (A/H1N1)',
                     'Seasonal Flu', 'Hong Kong flu (1968 pandemic)', 'Spanish flu (1918 pandemic)', 'COVID-19 (Wuhan Coronavirus)']
s.store_objects(pandemic_df=pandemic_df)

In [ ]:

s.store_objects(pandemic_df=pandemic_df)

In [ ]:

pandemic_df = s.load_object('pandemic_df')
covid19_df = s.load_object('covid19_df')
pandemic_df = pd.merge(left=pandemic_df, right=covid19_df, how='outer', left_index=True, right_index=True, suffixes=('_old', '_new'))
pandemic_df.columns = ['Transmission', 'R0', 'Case_fatality_ratio', 'Hospitalized_cases_sent_to_ICU', 'Asymptomatic_transmission',
                       'Year_vaccine_available', 'R0_high', 'R0_low', 'CFR_high', 'CFR_low', 'deaths_per_day_global_rounded',
                       'total_news_mentions_millions_data_retrieved_3rd_mar_2020', 'news_mentions_per_death', 'CFR_new',
                       'R0_new', 'annual_global_fatalities_latest_data_year', 'new_cases_per_year_latest_data_year',
                       'days_of_outbreak', 'fatalty_notes', 'source', 'url']
columns_list = sorted([cn for cn in pandemic_df.columns if ('_low' in cn) or ('_high' in cn) or ('_new' in cn)])
num_regex = re.compile(r'[^0-9.]+')
for disease_name, row_series in pandemic_df[columns_list].iterrows():
    CFR_high = row_series['CFR_high']
    CFR_low = row_series['CFR_low']
    CFR_new = row_series['CFR_new']
    if str(CFR_new) != 'nan':
        CFR_new = float(num_regex.sub('', str(CFR_new)))
    if str(CFR_high) == 'nan':
        pandemic_df.loc[disease_name, 'CFR_high'] = CFR_new
    if str(CFR_low) == 'nan':
        pandemic_df.loc[disease_name, 'CFR_low'] = CFR_new
    if str(CFR_new) != 'nan':
        if CFR_new < CFR_low:
            pandemic_df.loc[disease_name, 'CFR_low'] = CFR_new
        elif CFR_new > CFR_high:
            pandemic_df.loc[disease_name, 'CFR_high'] = CFR_new
    R0_high = row_series['R0_high']
    R0_low = row_series['R0_low']
    R0_new = row_series['R0_new']
    if str(R0_new) != 'nan':
        R0_new = float(num_regex.sub('', str(R0_new)))
    if str(R0_high) == 'nan':
        pandemic_df.loc[disease_name, 'R0_high'] = R0_new
    if str(R0_low) == 'nan':
        pandemic_df.loc[disease_name, 'R0_low'] = R0_new
    if str(R0_new) != 'nan':
        if R0_new < R0_low:
            pandemic_df.loc[disease_name, 'R0_low'] = R0_new
        elif R0_new > R0_high:
            pandemic_df.loc[disease_name, 'R0_high'] = R0_new
columns_list = ['Transmission', 'R0', 'Case_fatality_ratio', 'Hospitalized_cases_sent_to_ICU', 'Asymptomatic_transmission',
                'Year_vaccine_available', 'R0_high', 'R0_low', 'CFR_high', 'CFR_low', 'deaths_per_day_global_rounded',
                'total_news_mentions_millions_data_retrieved_3rd_mar_2020', 'news_mentions_per_death',
                'annual_global_fatalities_latest_data_year', 'new_cases_per_year_latest_data_year',
                'days_of_outbreak', 'fatalty_notes', 'source', 'url']
pandemic_df = pandemic_df[columns_list]
pandemic_df.sample(5).T.sample(6).T

In [ ]:

covid19_df = s.load_object('covid19_df')
covid19_df.index = ['Tuberculosis', 'Hepatitis B', 'Pneumonia', 'HIV ', 'Malaria ', 'Shigellosis', 'Rotavirus', 'Seasonal Flu',
                    'Norovirus', 'Whooping Cough', 'Typhoid', 'Cholera', 'Meningitis', 'Measles', 'Rabies ', 'Yellow Fever',
                    'Leishmaniasis', 'Echinococcosis', 'COVID-19 (Wuhan Coronavirus)', 'Dengue Fever', 'Hepatitis A',
                    'Chicken Pox', 'Sleeping Sickness', 'Ebola', 'SARS', 'MERS']
s.store_objects(covid19_df=covid19_df)

In [ ]:

pandemic_df = s.load_object('pandemic_df')
covid19_df = s.load_object('covid19_df')
check_for_typos(left_list=covid19_df.index.tolist(), right_list=pandemic_df.index.tolist(), verbose=False).sort_values('left_item')

In [ ]:

def get_page_soup(page_url):
    with urllib.request.urlopen(page_url) as response:
        page_html = response.read()
    page_soup = bs(page_html, 'html.parser')
    
    return page_soup

In [ ]:

tables_url = 'https://www.zorinaq.com/pub/ncov-comparison.html'
ncov_path = os.path.join(s.data_folder, 'html', 'ncov-comparison.html')
#page_soup = get_page_soup(tables_url)
tables_list = get_page_tables(tables_url)

In [ ]:

import numpy as np

pandemic_df = tables_list[0].copy()
pandemic_df.columns = ['Disease', 'Transmission', 'R0', 'Case_fatality_ratio', 'Hospitalized_cases_sent_to_ICU',
                       'Asymptomatic_transmission', 'Year_vaccine_available']

def f(x):
    result = re.sub(r' *\(', ' (', str(x))
    
    return result

pandemic_df.Disease = pandemic_df.Disease.map(f)

def f(x):
    result = re.sub(r'⁹', '', str(x))
    
    return result

pandemic_df.Transmission = pandemic_df.Transmission.map(f)
pandemic_df['R0_high'] = np.nan

def f(x):
    result = str(x).split('-')[-1]
    result = re.sub(r'[^0-9.]+', '', result)
    
    return float(result)

pandemic_df['R0_high'] = pandemic_df.R0.map(f)
pandemic_df['R0_low'] = np.nan

def f(x):
    result_list = str(x).split('-')
    if len(result_list) < 3:
        result = result_list[0]
    else:
        result = result_list[1].split(' ')[-1]
    result = re.sub(r'[^0-9.]+', '', result)
    
    return float(result)

pandemic_df['R0_low'] = pandemic_df.R0.map(f)
pandemic_df['CFR_high'] = np.nan

def f(x):
    result = str(x).split('-')[-1]
    result = re.sub(r'[^0-9.]+', '', result)
    
    return float(result)

pandemic_df['CFR_high'] = pandemic_df.Case_fatality_ratio.map(f)
pandemic_df['CFR_low'] = np.nan

def f(x):
    result_list = str(x).split('-')
    result = result_list[0]
    result = re.sub(r'[^0-9.]+', '', result)
    
    return float(result)

pandemic_df['CFR_low'] = pandemic_df.Case_fatality_ratio.map(f)
#print(pandemic_df.R0.unique().tolist())
#print(pandemic_df.R0.map(f).unique().tolist())
s.store_objects(pandemic_df=pandemic_df)
pandemic_df.sample(5).T

In [ ]:

date_format = '%b %d, %Y'
usa_df.Date = usa_df.Date.map(lambda x: datetime.strptime('{}, 2020'.format(x), date_format))
s.store_objects(usa_df=usa_df)
s.save_dataframes(usa_df=usa_df)

In [ ]:

usa_df = s.load_object('usa_df')
usa_df.set_index('Date', drop=True, inplace=True)
italy_df = s.load_object('italy_df')
italy_df.set_index('Date', drop=True, inplace=True)
columns_list = ['Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative']
tracking_df = usa_df[columns_list].merge(italy_df[columns_list], how='outer',
                                         left_index=True, right_index=True, suffixes=('_usa', '_italy'))
s.store_objects(tracking_df=tracking_df)
s.save_dataframes(include_index=True, tracking_df=tracking_df)

In [428]:

china_df = s.load_object('china_df')
china_df.Date = china_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
china_df.set_index('Date', drop=True, inplace=True)
czech_df = s.load_object('czech_df')
czech_df.Date = czech_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
czech_df.set_index('Date', drop=True, inplace=True)
columns_list = list(set(china_df.columns).intersection(set(czech_df.columns)))
merge_df = china_df[columns_list].merge(czech_df[columns_list], how='outer',
                                        left_index=True, right_index=True,
                                        suffixes=('_china', '_czech'))
print('czech_df:', merge_df.index.has_duplicates)


germany_df = s.load_object('germany_df')
germany_df.Date = germany_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
germany_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(germany_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_germany'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany']
print('germany_df:', merge_df.index.has_duplicates)


hong_kong_df = s.load_object('hong_kong_df')
hong_kong_df.Date = hong_kong_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
hong_kong_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(hong_kong_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_hong_kong'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong']
print('hong_kong_df:', merge_df.index.has_duplicates)


iran_df = s.load_object('iran_df')
iran_df.Date = iran_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
iran_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(iran_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_iran'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong', 'Confirmed_New_iran']
print('iran_df:', merge_df.index.has_duplicates)


italy_df = s.load_object('italy_df')
italy_df.Date = italy_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
italy_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(italy_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_italy'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy']
print('italy_df:', merge_df.index.has_duplicates)


japan_df = s.load_object('japan_df')
japan_df.Date = japan_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
japan_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(japan_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_japan'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan']
print('japan_df:', merge_df.index.has_duplicates)


netherlands_df = s.load_object('netherlands_df')
netherlands_df.Date = netherlands_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
netherlands_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(netherlands_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_netherlands'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands']
print('netherlands_df:', merge_df.index.has_duplicates)


singapore_df = s.load_object('singapore_df')
singapore_df.Date = singapore_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
singapore_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(singapore_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_singapore'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands', 'Confirmed_New_singapore']
print('singapore_df:', merge_df.index.has_duplicates)


slovakia_df = s.load_object('slovakia_df')
slovakia_df.Date = slovakia_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
slovakia_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(slovakia_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_slovakia'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands',
                    'Confirmed_New_singapore', 'Confirmed_New_slovakia']
print('slovakia_df:', merge_df.index.has_duplicates)


south_korea_df = s.load_object('south_korea_df')
south_korea_df.Date = south_korea_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
south_korea_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(south_korea_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_south_korea'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands',
                    'Confirmed_New_singapore', 'Confirmed_New_slovakia', 'Confirmed_New_south_korea']
print('south_korea_df:', merge_df.index.has_duplicates)


spain_df = s.load_object('spain_df')
spain_df.Date = spain_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
spain_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(spain_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_spain'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands',
                    'Confirmed_New_singapore', 'Confirmed_New_slovakia', 'Confirmed_New_south_korea', 'Confirmed_New_spain']
print('spain_df:', merge_df.index.has_duplicates)


uk_df = s.load_object('uk_df')
uk_df.Date = uk_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
uk_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(uk_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_uk'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong',
                    'Confirmed_New_iran', 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands',
                    'Confirmed_New_singapore', 'Confirmed_New_slovakia', 'Confirmed_New_south_korea',
                    'Confirmed_New_spain', 'Confirmed_New_uk']
print('uk_df:', merge_df.index.has_duplicates)


usa_df = s.load_object('usa_df')
usa_df.Date = usa_df.Date.map(lambda ts: date(ts.year, ts.month, ts.day))
usa_df.set_index('Date', drop=True, inplace=True)
merge_df = merge_df.merge(usa_df[columns_list], how='outer', left_index=True,
                          right_index=True, suffixes=('_merge', '_usa'))
merge_df.columns = ['Confirmed_New_china', 'Confirmed_New_czech', 'Confirmed_New_germany', 'Confirmed_New_hong_kong', 'Confirmed_New_iran',
 'Confirmed_New_italy', 'Confirmed_New_japan', 'Confirmed_New_netherlands', 'Confirmed_New_singapore',
 'Confirmed_New_slovakia', 'Confirmed_New_south_korea', 'Confirmed_New_spain', 'Confirmed_New_uk', 'Confirmed_New_usa']
print('usa_df:', merge_df.index.has_duplicates)

czech_df: False
germany_df: False
hong_kong_df: False
iran_df: False
italy_df: False
japan_df: False
netherlands_df: False
singapore_df: False
slovakia_df: False
south_korea_df: False
spain_df: False
uk_df: False
usa_df: False


In [25]:

countries_list = ['Thailand', 'China', 'Japan', 'South Korea', 'Laos', 'Cambodia', 'Singapore', 'the Philippines', 'Hong Kong',
                  'Vietnam', 'Malaysia']
df_set = set(['{}_df'.format('_'.join(country.lower().split(' '))) for country in countries_list])

In [26]:

df_set

{'south_korea_df', 'hong_kong_df', 'japan_df', 'the_philippines_df', 'singapore_df', 'malaysia_df', 'china_df', 'vietnam_df', 'thailand_df', 'laos_df', 'cambodia_df'}

In [78]:

created_set = set()
df_name_list = []
for file_name in os.listdir(s.saves_pickle_folder):
    if file_name.endswith('_df.pickle'):
        df_name = file_name.split('.')[0]
        df_name_list.append(df_name)
        if df_name in df_set:
            created_set.add(df_name)
created_set

{'south_korea_df', 'hong_kong_df', 'japan_df', 'thailand_df', 'malaysia_df', 'china_df', 'vietnam_df', 'singapore_df'}

In [9]:

usa_df = s.load_object('usa_df')

# Reporting Country/Territory/Area Total_confirmed_cases Total_confirmed_new_cases Total_deaths Total_new_deaths
# Transmission_classification Days_since_last_reported_case

# Timestamp('2020-03-26 00:00:00')
# United States of America 63570 11656 884 211 Local transmission 0
usa_df.loc[38, 'Confirmed_Cumulative'] = 63570
usa_df.loc[38, 'Confirmed_New'] = 11656
usa_df.loc[38, 'Deaths_Cumulative'] = 884
usa_df.loc[38, 'Deaths_New'] = 211

In [11]:

# Timestamp('2020-03-27 00:00:00')
# United States of America 68334 4764 991 107 Local transmission 0
usa_df.loc[39, 'Confirmed_Cumulative'] = 68334
usa_df.loc[39, 'Confirmed_New'] = 4764
usa_df.loc[39, 'Deaths_Cumulative'] = 991
usa_df.loc[39, 'Deaths_New'] = 107

In [16]:

# United States of America 85228 16894 1243 252 Local transmission 0
columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative']
row_dict = {'Date': pd.Timestamp('2020-03-28 00:00:00'), 'Confirmed_New': 16894,
            'Confirmed_Cumulative': 85228, 'Deaths_New': 252, 'Deaths_Cumulative': 1243}
usa_df = pd.concat([usa_df, pd.DataFrame([row_dict], columns=columns_list, index=[40])])

In [ ]:

# United States of America ??? ??? ??? ??? Local transmission 0
columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative']
row_dict = {'Date': pd.Timestamp('2020-03-29 00:00:00'), 'Confirmed_New': xxxxxx,
            'Confirmed_Cumulative': xxxxxx, 'Deaths_New': xxxxxx, 'Deaths_Cumulative': xxxxxx}
usa_df = pd.concat([usa_df, pd.DataFrame([row_dict], columns=columns_list, index=[41])])

In [18]:

s.store_objects(usa_df=usa_df)

Pickling to D:\Documents\Repositories\notebooks\covid19\saves\pickle\usa_df.pickle


In [17]:

columns_list = ['Date', 'Confirmed_New', 'Confirmed_Cumulative', 'Deaths_New', 'Deaths_Cumulative']
usa_df[columns_list]

,Date,Confirmed_New,Confirmed_Cumulative,Deaths_New,Deaths_Cumulative
0,2020-01-21,1.0,1.0,0,0
1,2020-01-24,1.0,2.0,0,0
2,2020-01-25,1.0,3.0,0,0
3,2020-01-26,2.0,5.0,0,0
4,2020-01-30,1.0,6.0,0,0
5,2020-01-31,1.0,7.0,0,0
6,2020-02-01,1.0,8.0,0,0
7,2020-02-02,3.0,11.0,0,0
8,2020-02-05,1.0,12.0,0,0
9,2020-02-20,2.0,14.0,0,0


In [3]:

asian_list = ['south_korea_df', 'hong_kong_df', 'japan_df', 'thailand_df', 'malaysia_df', 'china_df', 'vietnam_df',
              'singapore_df', 'philippines_df']
cumulative_list = ['china_df', 'czech_df', 'germany_df', 'hong_kong_df', 'iran_df', 'italy_df', 'japan_df', 'netherlands_df',
                   'singapore_df', 'slovakia_df', 'south_korea_df', 'spain_df', 'uk_df', 'usa_df']
for df_name in set(asian_list + cumulative_list):
    df = s.load_object(df_name)
    df.columns = [re.sub('Recoveries_Cumulative', 'Recovered_Cumulative', cn) for cn in df.columns]
    kwargs = {df_name: df}
    s.store_objects(**kwargs)

Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\slovakia_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\iran_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\usa_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\italy_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\germany_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\thailand_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\hong_kong_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\czech_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\singapore_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\saves\pickle\china_df.pickle
Pickling to D:\Documents\Repositories\notebooks\StatsByCountry\save

In [18]:

states_stats_df = s.load_object('states_stats_df')
state_list = ['Arkansas', 'Iowa', 'Nebraska', 'North Dakota', 'South Dakota']
mask_series = states_stats_df.index.isin(state_list)
abbrev_dict = states_stats_df[mask_series].State_Abbreviation.to_dict()
abbrev_list = [abbrev_dict[state_name] for state_name in state_list]
abbrev_list

KeyError: 'Arkansas'

In [20]:

states_stats_df.index

Index(['Alabama', 'Alaska', 'Arizona', 'California', 'Colorado', 'Connecticut',
       'Delaware', 'Florida', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Kansas', 'Louisiana', 'Maine', 'Maryland', 'Michigan', 'Minnesota',
       'Mississippi', 'Montana', 'Nevada', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'Ohio', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'Tennessee', 'Texas', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin'],
      dtype='object')